In [1]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [359]:
df = pd.read_csv('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2020-10-19-oncohemato/compare/compare.revised_intermediate_test.tsv', sep="\t")

In [382]:
def revised_df(df, min_threshold_include=0.7, min_threshold_discard=0.7, remove_faulty=True, drop_samples=True, drop_positions=True):
    if remove_faulty == True:

        uncovered_positions = df.iloc[:,3:].apply(lambda x: sum((x == '!'))/len(x), axis=1)
        heterozygous_positions = df.iloc[:,3:].apply(lambda x: sum([i not in ['!',0,1, '0', '1'] for i in x.values])/len(x), axis=1)
        report_position = pd.DataFrame({'Position': df.Position, 'uncov_fract': uncovered_positions, 'htz_frac': heterozygous_positions, 'faulty_frac': uncovered_positions + heterozygous_positions})
        faulty_positions = report_position['Position'][report_position.faulty_frac >= min_threshold_discard].tolist()


        uncovered_samples = df.iloc[:,3:].apply(lambda x: sum((x == '!'))/len(x), axis=0)
        heterozygous_samples = df.iloc[:,3:].apply(lambda x: sum([i not in ['!',0,1, '0', '1'] for i in x.values])/len(x), axis=0)
        report_samples = pd.DataFrame({'sample': df.iloc[:,3:].columns, 'uncov_fract': uncovered_samples, 'htz_frac': heterozygous_samples, 'faulty_frac': uncovered_samples + heterozygous_samples})
        faulty_samples = report_samples['sample'][report_samples.faulty_frac >= min_threshold_discard].tolist()

        if drop_positions == True:
            df = df[~df.Position.isin(faulty_positions)]
        if drop_samples == True:
            df = df.drop(faulty_samples, axis=1)

        print('Faulty positions:\n{} \n\nFaulty samples:\n{}'.format(("\n").join(faulty_positions), ("\n").join(faulty_samples)))

    df = df.replace('!', 0)
    df.iloc[:,3:] = df.iloc[:,3:].astype(float)
    f = lambda x: 1 if x >= min_threshold_include else 0 # IF HANDLE HETEROZYGOUS CHANGE THIS 0 for X or 0.5
    df.iloc[:,3:] = df.iloc[:,3:].applymap(f)
    df.N = df.apply(lambda x: sum(x[3:]), axis=1)
    df = df[df.N > 0]
    return df

In [383]:
revised_df(df, min_threshold_include=0.7, min_threshold_discard=0.7, remove_faulty=True, drop_samples=True, drop_positions=True)

Faulty positions:
NC_045512.2|C|6286|T 

Faulty samples:
20379679


,Position,N,Samples,20379900,20384092,20392100,20392110
0,NC_045512.2|A|1987|G,4,"20379679,20379900,20384092,20392100",1,1,1,1
1,NC_045512.2|A|23403|G,4,"20379679,20379900,20384092,20392100",1,1,1,1
2,NC_045512.2|A|29407|G,4,"20379679,20379900,20384092,20392100",1,1,1,1
4,NC_045512.2|C|14408|T,3,"20379679,20379900,20384092,20392100",1,0,1,1
5,NC_045512.2|C|18129|T,3,"20379679,20379900,20384092,20392100",1,0,1,1
6,NC_045512.2|C|2102|T,3,"20379679,20379900,20384092,20392100",1,0,1,1
7,NC_045512.2|C|22208|T,3,"20379679,20379900,20384092,20392100",1,0,1,1
8,NC_045512.2|C|22227|T,3,"20379679,20379900,20384092,20392100",1,0,1,1
9,NC_045512.2|C|241|T,3,"20379679,20379900,20384092,20392100",1,0,1,1
10,NC_045512.2|C|26801|G,3,"20379679,20379900,20384092,20392100",1,0,1,1


In [375]:
df.head()

,Position,N,Samples,20379679,20379900,20384092,20392100,20392110
0,NC_045512.2|A|1987|G,4,"20379679,20379900,20384092,20392100",!,0.96,1,1,1
1,NC_045512.2|A|23403|G,4,"20379679,20379900,20384092,20392100",!,1.00,1,1,1
2,NC_045512.2|A|29407|G,4,"20379679,20379900,20384092,20392100",!,1.00,1,1,1
3,NC_045512.2|C|12115|T,1,20384092,!,0.00,0,0,0
4,NC_045512.2|C|14408|T,4,"20379679,20379900,20384092,20392100",!,1.00,!,1,1


In [376]:
uncovered_positions = df.iloc[:,3:].apply(lambda x: (x.values == '!').sum()/len(x), axis=1)
heterozygous_positions = df.iloc[:,3:].apply(lambda x: sum([i not in ['!',0,1, '0', '1'] for i in x.values])/len(x), axis=1)
report_position = pd.DataFrame({'Position': df.Position, 'uncov_fract': uncovered_positions, 'htz_frac': heterozygous_positions, 'faulty_frac': uncovered_positions + heterozygous_positions})


In [381]:
df.iloc[:,3:].apply(lambda x: sum((x == '!'))/len(x), axis=1)

0     0.2
1     0.2
2     0.2
3     0.2
4     0.4
5     0.4
6     0.4
7     0.4
8     0.4
9     0.4
10    0.4
11    0.2
12    0.2
13    0.2
14    0.6
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
dtype: float64

In [197]:
uncovered_samples = df.iloc[:,3:].apply(lambda x: (x.values == '!').sum()/len(x), axis=0)
heterozygous_samples = df.iloc[:,3:].apply(lambda x: sum([i not in ['!',0,1, '0', '1'] for i in x.values])/len(x), axis=0)
report_samples = pd.DataFrame({'sample': df.iloc[:,3:].columns, 'uncov_fract': uncovered_samples, 'htz_frac': heterozygous_samples, 'faulty_frac': uncovered_samples + heterozygous_samples})

In [221]:
faulty_samples = report_samples['sample'][report_samples.faulty_frac >= 0.5]
faulty_samples.tolist()

['20379679', '20384092']

In [201]:
faulty_positions = report_position['Position'][report_position.faulty_frac >= 0.5]
faulty_positions.tolist()

['NC_045512.2|A|1987|G',
 'NC_045512.2|C|14408|T',
 'NC_045512.2|C|18129|T',
 'NC_045512.2|C|2102|T',
 'NC_045512.2|C|22208|T',
 'NC_045512.2|C|22227|T',
 'NC_045512.2|C|241|T',
 'NC_045512.2|C|26801|G',
 'NC_045512.2|C|3037|T',
 'NC_045512.2|C|6286|T',
 'NC_045512.2|G|24197|T']

In [233]:
#df.filter(faulty_samples.tolist())

In [234]:
df.drop(['20379679', '20384092'], axis=1)[~(df.Position.isin(faulty_positions.tolist()))]

,Position,N,Samples,20379900,20392100
1,NC_045512.2|A|23403|G,4,"20379679,20379900,20384092,20392100",1.0,1
2,NC_045512.2|A|29407|G,4,"20379679,20379900,20384092,20392100",1.0,1
3,NC_045512.2|C|12115|T,1,20384092,0.0,0
11,NC_045512.2|C|27944|T,4,"20379679,20379900,20384092,20392100",1.0,1
12,NC_045512.2|C|28932|T,4,"20379679,20379900,20384092,20392100",1.0,1
15,NC_045512.2|G|21255|C,4,"20379900,20384092,20392100",1.0,1
17,NC_045512.2|G|29645|T,4,"20379679,20379900,20384092,20392100",1.0,1
18,NC_045512.2|G|9802|T,4,"20379679,20379900,20384092,20392100",1.0,1
19,NC_045512.2|T|445|C,4,"20379679,20379900,20384092,20392100",1.0,1


In [313]:
df = df.replace('!', 0)

In [314]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
Position    20 non-null object
N           20 non-null int64
Samples     20 non-null object
20379679    20 non-null object
20379900    20 non-null float64
20384092    20 non-null object
20392100    20 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 1.2+ KB


In [315]:
df.iloc[:,3:] = df.iloc[:,3:].astype(float)

In [316]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
Position    20 non-null object
N           20 non-null int64
Samples     20 non-null object
20379679    20 non-null float64
20379900    20 non-null float64
20384092    20 non-null float64
20392100    20 non-null float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.2+ KB


In [329]:
f = lambda x: 1 if x >= 0.7 else 0

df.iloc[:,3:].applymap(f)

,20379679,20379900,20384092,20392100
0,0,1,1,1
1,0,1,1,1
2,0,1,1,1
3,0,0,1,0
4,0,1,0,1
5,0,1,0,1
6,0,1,0,1
7,0,1,0,1
8,0,1,0,1
9,0,1,0,1


In [334]:
df

,Position,N,Samples,20379679,20379900,20384092,20392100
0,NC_045512.2|A|1987|G,4,"20379679,20379900,20384092,20392100",0.0,0.96,1.0,1.0
1,NC_045512.2|A|23403|G,4,"20379679,20379900,20384092,20392100",0.0,1.00,1.0,1.0
2,NC_045512.2|A|29407|G,4,"20379679,20379900,20384092,20392100",0.0,1.00,1.0,1.0
3,NC_045512.2|C|12115|T,1,20384092,0.0,0.00,1.0,0.0
4,NC_045512.2|C|14408|T,4,"20379679,20379900,20384092,20392100",0.0,1.00,0.0,1.0
5,NC_045512.2|C|18129|T,4,"20379679,20379900,20384092,20392100",0.0,1.00,0.0,1.0
6,NC_045512.2|C|2102|T,4,"20379679,20379900,20384092,20392100",0.0,1.00,0.0,1.0
7,NC_045512.2|C|22208|T,4,"20379679,20379900,20384092,20392100",0.0,1.00,0.0,1.0
8,NC_045512.2|C|22227|T,4,"20379679,20379900,20384092,20392100",0.0,1.00,0.0,1.0
9,NC_045512.2|C|241|T,4,"20379679,20379900,20384092,20392100",0.0,1.00,0.0,1.0
